In [1]:
import pandas as pd


test_data = pd.read_csv(
    "../files/input/test_data.csv.zip",
    index_col=False,
    compression="zip",
)

train_data = pd.read_csv(
    "../files/input/train_data.csv.zip",
    index_col=False,
    compression="zip",
)

In [2]:
# - Renombre la columna "default payment next month" a "default".
test_data = test_data.rename(columns={'default payment next month': 'default'})
train_data = train_data.rename(columns={'default payment next month': 'default'})

print(test_data.columns)

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default'],
      dtype='object')


In [3]:
# - Remueva la columna "ID".
test_data=test_data.drop(columns=['ID'])
train_data=train_data.drop(columns=['ID'])

In [4]:
# - Elimine los registros con informacion no disponible.

import numpy as np

# Para train_data
train_data = train_data.loc[train_data["MARRIAGE"] != 0]
train_data = train_data.loc[train_data["EDUCATION"] != 0]
print(train_data["EDUCATION"].value_counts())
print(train_data["MARRIAGE"].value_counts())

# Para test_data (corregido)
test_data = test_data.loc[test_data["MARRIAGE"] != 0]
test_data = test_data.loc[test_data["EDUCATION"] != 0]
print(test_data["EDUCATION"].value_counts())
print(test_data["MARRIAGE"].value_counts())

EDUCATION
2    9756
1    7476
3    3396
5     187
4      98
6      40
Name: count, dtype: int64
MARRIAGE
2    11226
1     9502
3      225
Name: count, dtype: int64
EDUCATION
2    4268
1    3105
3    1477
5      93
4      25
6      11
Name: count, dtype: int64
MARRIAGE
2    4728
1    4153
3      98
Name: count, dtype: int64


In [5]:
# - Para la columna EDUCATION, valores > 4 indican niveles superiores
test_data['EDUCATION'] = test_data['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
train_data['EDUCATION'] = train_data['EDUCATION'].apply(lambda x: 4 if x > 4 else x)

print(train_data["EDUCATION"].value_counts())
print(test_data["EDUCATION"].value_counts())

EDUCATION
2    9756
1    7476
3    3396
4     325
Name: count, dtype: int64
EDUCATION
2    4268
1    3105
3    1477
4     129
Name: count, dtype: int64


In [6]:
# Divida los datasets en x_train, y_train, x_test, y_test.
x_train=train_data.drop(columns="default")
y_train=train_data["default"]


x_test=test_data.drop(columns="default")
y_test=test_data["default"]

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Columnas categóricas
categorical_features = ["SEX", "EDUCATION", "MARRIAGE"]

# Preprocesador: OneHotEncoding para categóricas, escalar numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features),  # One-hot encoding para categóricas
        ('num', MinMaxScaler(), 'passthrough')          # Escalado para numéricas
    ],
    remainder="passthrough"  # Mantener las columnas no especificadas
)

# Crear el pipeline con un preprocesador (por ejemplo, StandardScaler) y LogisticRegression
pipeline = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('scaler', StandardScaler(), x_train.columns)  # O cualquier transformación que necesites
    ])),
    ('classifier', LogisticRegression())  # Este es el clasificador que será optimizado
])



In [8]:

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.compose import ColumnTransformer

# Definir los hiperparámetros a optimizar para LogisticRegression
param_grid = {
    'classifier__C': [0.1, 1, 10],  # Regularización (tamaño del parámetro)
    'classifier__solver': ['lbfgs', 'liblinear'],  # Solucionadores
    'classifier__penalty': ['l2'],   # Penalización L2 (común en regresión logística)
    'classifier__max_iter': [100, 200]  # Número máximo de iteraciones
}

# Crear la función de puntuación
scorer = make_scorer(balanced_accuracy_score)

# Modelo con GridSearchCV
model = GridSearchCV(
    pipeline,
    param_grid,
    cv=10,                       # Validación cruzada de 10 divisiones
    scoring=scorer,              # Función de evaluación
    n_jobs=-1,                   # Usar todos los núcleos disponibles
    refit=True                   # Ajustar al mejor modelo
)

# Entrenar el modelo
model.fit(x_train, y_train)

# Mejor modelo y mejores hiperparámetros
print("Mejores parámetros:", model.best_params_)
print("Mejor puntuación:", model.best_score_)


Mejores parámetros: {'classifier__C': 10, 'classifier__max_iter': 100, 'classifier__penalty': 'l2', 'classifier__solver': 'lbfgs'}
Mejor puntuación: 0.6036920232769496


In [9]:
import pickle
import os

models_dir = '../files/models'
os.makedirs(models_dir, exist_ok=True)

with open("../files/models/model.pkl","wb") as file:
    pickle.dump(model,file)


In [10]:
import json
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score

def calculate_and_save_metrics(model, X_train, X_test, y_train, y_test):
    # Hacer predicciones
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calcular métricas para el conjunto de entrenamiento
    metrics_train = {
        'type': 'metrics',
        'dataset': 'train',
        'precision': precision_score(y_train, y_train_pred, zero_division=0),
        'balanced_accuracy': balanced_accuracy_score(y_train, y_train_pred),
        'recall': recall_score(y_train, y_train_pred, zero_division=0),
        'f1_score': f1_score(y_train, y_train_pred, zero_division=0)
    }

    # Calcular métricas para el conjunto de prueba
    metrics_test = {
        'type': 'metrics',
        'dataset': 'test',
        'precision': precision_score(y_test, y_test_pred, zero_division=0),
        'balanced_accuracy': balanced_accuracy_score(y_test, y_test_pred),
        'recall': recall_score(y_test, y_test_pred, zero_division=0),
        'f1_score': f1_score(y_test, y_test_pred, zero_division=0)
    }

    # Crear carpeta si no existe
    output_dir = '../files/output'
    os.makedirs(output_dir, exist_ok=True)

    # Guardar las métricas en un archivo JSON
    output_path = os.path.join(output_dir, 'metrics.json')
    with open(output_path, 'w') as f:  # Usar 'w' para comenzar con un archivo limpio
        f.write(json.dumps(metrics_train) + '\n')
        f.write(json.dumps(metrics_test) + '\n')

In [11]:
from sklearn.metrics import confusion_matrix
# Función para calcular las matrices de confusión y guardarlas
def calculate_and_save_confusion_matrices(model, X_train, X_test, y_train, y_test):
    # Hacer predicciones
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calcular matrices de confusión
    cm_train = confusion_matrix(y_train, y_train_pred)
    cm_test = confusion_matrix(y_test, y_test_pred)

    # Convertir las matrices de confusión en formato JSON
    def format_confusion_matrix(cm, dataset_type):
        return {
            'type': 'cm_matrix',
            'dataset': dataset_type,
            'true_0': {
                'predicted_0': int(cm[0, 0]),
                'predicted_1': int(cm[0, 1])
            },
            'true_1': {
                'predicted_0': int(cm[1, 0]),
                'predicted_1': int(cm[1, 1])
            }
        }

    metrics = [
        format_confusion_matrix(cm_train, 'train'),
        format_confusion_matrix(cm_test, 'test')
    ]

    # Guardar las matrices de confusión en el mismo archivo JSON
    output_path = '../files/output/metrics.json'
    with open(output_path, 'a') as f:  # Usar 'a' para agregar después de las métricas
        for metric in metrics:
            f.write(json.dumps(metric) + '\n')

# Función principal para ejecutar todo
def main(model, X_train, X_test, y_train, y_test):
    # Crear el directorio de salida si no existe
    import os
    os.makedirs('../files/output', exist_ok=True)

    # Calcular y guardar las métricas
    calculate_and_save_metrics(model, X_train, X_test, y_train, y_test)

    # Calcular y guardar las matrices de confusión
    calculate_and_save_confusion_matrices(model, X_train, X_test, y_train, y_test)

# Ejemplo de uso:
main(model, x_train, x_test, y_train, y_test)